In [35]:
import numpy as np
import scipy.linalg as sla

In [45]:
dataset1 = (
    np.array([
        [1, 2, -1, 0, 3, 2, 3, 1, 5],
        [1, 0, 0, -1, -1, -3, 0, -6, 1],
        [0, 1, -4, 1, 3, 1, 0, 1, 0],
        [1, 2, -3, 1, -1, 2, 5, 0, 4],
    ]),
    np.array([-1, -3, 1, -5, 2, -3, 1, -1, -1]),
    np.array([28.0, 59.0, -3.0, 3.0]),
    [0, 3, 4, 7],
)

In [37]:
def potentials(vector, inv_matrix):
    return vector.dot(inv_matrix)


def approx(pot, opp_idx, A, c):
    results = [0.0] * A.shape[1]
    all_pos = True
    for i in opp_idx:
        results[i] = pot.dot(A[:, i]) - c[i]
        if np.abs(results[i]) < 1e-9:
            results[i] = 0.0
        elif results[i] < 0:
            all_pos = False
    return results if not all_pos else None



In [48]:
def get_next_solution(dataset, B, x):
    A = dataset[0]
    c = dataset[1]
    b = dataset[2]
    idx = dataset[3]
    size = A.shape[1]
    opposite_idx = set(range(size))
    for i in idx:
        opposite_idx.remove(i)
    A_b = A[:, idx]
    c_b = c[idx]
    print(A_b)
    if B is None:
        B = sla.inv(A_b)
    
    pot = potentials(c_b, B)
    approximation = approx(pot, opposite_idx, A, c)
    if approximation is None:
        print(f"{x} is already the most optimized plan")
        return x
    print(approximation)

    j = 0.0
    for i in range(len(approximation)):
        if approximation[i] < 0:
            j = i
            break
    
    if j == 0.0:
        print("Some error occured")
        return None
    
    z = B.dot(A[:, j])
    
    pos_idx = []
    for i in range(len(z)):
        if z[i] > 0.0:
            pos_idx.append(i)
        
    print("z: ", z)
    
    if len(pos_idx) == 0:
        return None
    
    min_value = None
    
    for i in range(len(z)):
        value = x[idx[i]] / z[i]
        if min_value is None or value < min_value:
            min_value = value
            x_idx = idx[i]
            z_idx = i
    
    if min_value is None:
        print("Some error occured")
        return None
    
    new_x = x.copy()
    for i in range(len(z)):
        new_x[idx[i]] = x[idx[i]] - min_value * z[i]
        if np.abs(new_x[idx[i]]) < 1e-9:
            new_x[idx[i]] = 0.0
    new_x[j] = min_value
    idx.remove(x_idx)
    idx.append(j)
    sorted(idx)
    print(idx)
    M = np.eye(len(z))
    d = np.array([0.0] * len(z))
    for i in range(len(z)):
        d[i] = -z[i]
    d[z_idx] = 1
    d /= z[z_idx]
    M[:, z_idx] = d
    print(M)
    B = M.dot(B)
    return new_x
    

def solve(dataset, init_plan, max_iter=100):
    curr_plan = init_plan
    B = None
    for i in range(max_iter):
        new_plan = get_next_solution(dataset, B, curr_plan)
        if curr_plan is None:
            print("No solution as function has no upper bound")
            break
        plans_equal = True
        for i in range(len(curr_plan)):
            if np.abs(new_plan[i] - curr_plan[i]) != 0:
                plans_equal = False
                break
        
        if plans_equal:
            print(curr_plan)
            break
        else:
            curr_plan = new_plan
    print(curr_plan)


In [49]:
solve(dataset1, np.array([2, 0, 0, 7, 6, 0, 0, 8, 0]))

[[ 0  3  1  2]
 [-1 -1 -6  0]
 [ 1  3  1  1]
 [ 1 -1  0  2]]
[0.8064516129032269, 0.0, 13.693548387096778, 0.0, 0.0, -0.4354838709677389, -7.080645161290317, 0.0, 1.983870967741943]
z:  [-0.0483871  -0.14516129  0.53225806  0.9516129 ]
[4, 7, 1, 5]
[[-20.66666667   0.           0.           0.        ]
 [ -3.           1.           0.           0.        ]
 [ 11.           0.           1.           0.        ]
 [ 19.66666667   0.           0.           1.        ]]
[[ 3  1  2  2]
 [-1 -6  0 -3]
 [ 3  1  1  1]
 [-1  0  2  2]]
[3.999999999999993, 0.0, 41.999999999999936, -8.999999999999979, 0.0, 0.0, -2.000000000000007, 0.0, 20.999999999999957]
z:  [ -3.          11.          19.66666667 -20.66666667]
[7, 1, 5, 3]
[[-0.33333333  0.          0.          0.        ]
 [ 3.66666667  1.          0.          0.        ]
 [ 6.55555556  0.          1.          0.        ]
 [-6.88888889  0.          0.          1.        ]]
[[ 1  2  2  0]
 [-6  0 -3 -1]
 [ 1  1  1  1]
 [ 0  2  2  1]]
[0.999999999